In [ ]:
import time
import os
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms.optimizers import COBYLA ,SPSA
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# -------------------------
# Evidence & results setup
# -------------------------
def setup_evidence_collection():
    evidence_dir = "evidence"
    logs_dir = os.path.join(evidence_dir, "logs")
    screenshots_dir = os.path.join(evidence_dir, "screenshots")
    os.makedirs(evidence_dir, exist_ok=True)
    os.makedirs(logs_dir, exist_ok=True)
    os.makedirs(screenshots_dir, exist_ok=True)
    job_ids_file = os.path.join(evidence_dir, "job_ids.csv")
    if not os.path.exists(job_ids_file):
        with open(job_ids_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['timestamp', 'backend', 'job_id', 'mode', 'shots', 'notes'])
    return job_ids_file, logs_dir, screenshots_dir

def record_job(job_ids_file, backend_name, job_id, mode, shots, notes=""):
    timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
    with open(job_ids_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, backend_name, job_id, mode, shots, notes])
    print(f"Recorded job: {job_id} on {backend_name}")

job_ids_file, logs_dir, screenshots_dir = setup_evidence_collection()

# -------------------------
# Problem data
# -------------------------
data = {
  "problem_description": {"constraints": {"stops_per_trip": 3}},
  "locations": {
    "hospital": {"name":"Central Hospital","coordinates":{"latitude":29.99512653425452,"longitude":31.68462840171934}},
    "patients":[
      {"id":"DT","coordinates":{"latitude":30.000417586266437,"longitude":31.73960813272627}},
      {"id":"GR","coordinates":{"latitude":30.011344405285193,"longitude":31.747827362371993}},
      {"id":"R2","coordinates":{"latitude":30.030388325206854,"longitude":31.669231198639675}},
      {"id":"R3_2","coordinates":{"latitude":30.030940768851426,"longitude":31.688371339937028}},
      {"id":"IT","coordinates":{"latitude":30.01285635906825,"longitude":31.693811715848444}}
    ]
  }
}

hospital = data["locations"]["hospital"]["coordinates"]
patients_data = data["locations"]["patients"]
ids = [p["id"] for p in patients_data]
points = ["H"] + ids   # node names, index 0 = hospital
n = len(points)       # should be 6

# distance dictionary (explicit)
distance_dict = {
    'H': {'H': 0.0, 'DT': 8.6285, 'GR': 11.4958, 'R2': 9.4454, 'R3_2': 10.8524, 'IT': 9.6724},
    'DT': {'H': 14.1936, 'DT': 0.0, 'GR': 2.3608, 'R2': 10.922, 'R3_2': 9.238, 'IT': 9.4305},
    'GR': {'H': 17.7848, 'DT': 7.7452, 'GR': 0.0, 'R2': 11.8083, 'R3_2': 10.1243, 'IT': 10.478},
    'R2': {'H': 11.8644, 'DT': 19.6719, 'GR': 15.6608, 'R2': 0.0, 'R3_2': 11.5718, 'IT': 11.5385},
    'R3_2': {'H': 7.3427, 'DT': 12.1719, 'GR': 10.0531, 'R2': 4.0711, 'R3_2': 0.0, 'IT': 5.9308},
    'IT': {'H': 9.2692, 'DT': 9.3984, 'GR': 12.2657, 'R2': 7.318, 'R3_2': 8.725, 'IT': 0.0}
}

# precompute numpy dist matrix
dist = np.zeros((n, n))
for i, a in enumerate(points):
    for j, b in enumerate(points):
        dist[i, j] = distance_dict[a][b]

# -------------------------
# Problem parameters
# -------------------------
n_trips = 2
max_stops = data["problem_description"]["constraints"]["stops_per_trip"]
patient_count = len(ids)
global_capacity_rhs = min(patient_count, n_trips * max_stops)  # 5

# -------------------------
# Build QuadraticProgram (only binary routing vars)
# -------------------------
qp = QuadraticProgram("vehicle_routing_30qubits")

# add x_i_j binary for i != j
for i in range(n):
    for j in range(n):
        if i != j:
            qp.binary_var(name=f"x_{i}_{j}")

# objective: minimize total distance
linear = {}
for i in range(n):
    for j in range(n):
        if i != j:
            linear[f"x_{i}_{j}"] = dist[i, j]
qp.minimize(linear=linear)

# -------------------------
# Add constraints
# -------------------------
# 1) Each patient (non-hospital) incoming == 1 and outgoing == 1
for k in range(1, n):
    incoming = {f"x_{i}_{k}": 1 for i in range(n) if i != k}
    outgoing = {f"x_{k}_{j}": 1 for j in range(n) if j != k}
    qp.linear_constraint(linear=incoming, sense='==', rhs=1, name=f"incoming_eq_{k}")
    qp.linear_constraint(linear=outgoing, sense='==', rhs=1, name=f"outgoing_eq_{k}")

# 2) Hospital departures == n_trips, returns == n_trips
qp.linear_constraint(linear={f"x_0_{j}": 1 for j in range(1, n)}, sense='==', rhs=n_trips, name='departures_from_hospital')
qp.linear_constraint(linear={f"x_{i}_0": 1 for i in range(1, n)}, sense='==', rhs=n_trips, name='returns_to_hospital')

# 3) Capacity: total patient incoming visits <= global_capacity_rhs
unique_patient_incoming = {f"x_{i}_{k}" for k in range(1, n) for i in range(n) if i != k}
qp.linear_constraint(linear={v: 1 for v in unique_patient_incoming}, sense='<=', rhs=global_capacity_rhs, name='global_capacity')

# 4) Prevent single-patient H->k->H mini-loops (no singleton trips)
#    If both x_0_k and x_k_0 are 1 simultaneously, that's a singleton trip. We forbid that by <= 1.
for k in range(1, n):
    qp.linear_constraint(linear={f"x_0_{k}": 1, f"x_{k}_0": 1}, sense='<=', rhs=1, name=f"no_singleton_{k}")

print("Quadratic program built.")
print(f"Binary variables (should be 30): {qp.get_num_vars()}")
print(f"Linear constraints: {len(qp.linear_constraints)}")

# -------------------------
# Convert to QUBO (penalty handles inequalities)
# -------------------------
penalty_scale = 170
q2q = QuadraticProgramToQubo(penalty=penalty_scale)
qubo = q2q.convert(qp)
print(f"QUBO created with penalty={penalty_scale}")
print(f"QUBO binary variables: {qubo.get_num_vars()}")

# -------------------------
# IBM Quantum backend selection
# -------------------------
service = QiskitRuntimeService()
backends = service.backends(simulator=False, operational=True)
backends_sorted = sorted(backends, key=lambda b: b.status().pending_jobs if b.status() else 9999)
backend = backends_sorted[0]
backend_name = backend.name
print(f"Using backend: {backend_name} (pending jobs: {backend.status().pending_jobs})")
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=1)

# -------------------------
# QAOA setup
# -------------------------
loss_history = []
def callback(eval_count, parameters, mean, std):
    loss_history.append(float(np.real(mean)))
    print(f"Iteration {eval_count}: Energy = {loss_history[-1]}")

optimizer = COBYLA(maxiter=200)
reps = 3
shots = 2048

# -------------------------
# Run QAOA (MinimumEigenOptimizer)
# -------------------------
t0 = time.time()
print("Running QAOA (MinimumEigenOptimizer) ...")

from qiskit_optimization.minimum_eigensolvers import QAOA
with Session(backend=backend) as session:
    sampler = Sampler()
    qaoa = QAOA(
        sampler=sampler,
        optimizer=optimizer,
        reps=reps,
        initial_point=[0.1] * (2*reps),
        callback=callback,
        pass_manager=pass_manager
    )
    meo = MinimumEigenOptimizer(qaoa)
    result = meo.solve(qubo)

    # try to record job id if available from sampler
    job_id = getattr(sampler, "last_job", None)
    if job_id is not None:
        try:
            jid = job_id.job_id()
        except Exception:
            jid = str(job_id)
    else:
        jid = "unknown_job_id"
    record_job(job_ids_file, backend_name, jid, "physical", shots, "QAOA 30-qubit VRP")

t1 = time.time()
elapsed = t1 - t0

if result is None:
    raise RuntimeError("No result obtained from optimizer. Aborting.")

print(f"Finished in {elapsed:.3f} s. Status: {result.status}")
print(f"Objective value: {result.fval:.6f}")

# -------------------------
# Extract solution and active edges
# -------------------------
# result.variables_dict is a mapping varname -> value
vars_dict = getattr(result, "variables_dict", None)
if vars_dict is None:
    # fallback: try result.x and map to variable names from qubo
    try:
        var_names = [v.name for v in qubo.variables]
        vars_dict = {name: float(val) for name, val in zip(var_names, result.x)}
    except Exception:
        raise RuntimeError("Cannot extract result variables.")

active_edges = []
for var, val in vars_dict.items():
    if var.startswith("x_") and abs(val - 1) < 1e-6:
        _, i_str, j_str = var.split("_")
        i = int(i_str); j = int(j_str)
        active_edges.append((i, j))

print("\nActive edges:")
for (i, j) in active_edges:
    print(f"  {points[i]} -> {points[j]}")

# -------------------------
# Route reconstruction
# -------------------------
def reconstruct_routes(edges, start_node=0):
    edges_copy = edges.copy()
    routes = []
    while edges_copy:
        route = [start_node]
        cur = start_node
        visited_nodes = set()
        while True:
            next_nodes = [j for (i,j) in edges_copy if i == cur]
            if not next_nodes:
                break
            nxt = next_nodes[0]
            route.append(nxt)
            edges_copy.remove((cur, nxt))
            cur = nxt
            # avoid infinite loops
            if len(route) > len(points) + 5:
                break
            if cur == start_node:
                break
        if len(route) > 1:
            # ensure closed route ends at hospital
            if route[-1] != start_node:
                route.append(start_node)
            routes.append(route)
        else:
            break
    return routes

routes = reconstruct_routes(active_edges, start_node=0)
print("\nReconstructed routes (indices -> names):")
for r in routes:
    print("  " + " -> ".join([points[i] for i in r]))

# -------------------------
# Compute trip distances and build trip_details
# -------------------------
total_distance = 0.0
trip_details = {}
for idx, route in enumerate(routes, start=1):
    route_nodes = route
    # ensure return to hospital
    if route_nodes[-1] != 0:
        route_nodes = route_nodes + [0]
    route_points = [points[i] for i in route_nodes]
    dist_sum = 0.0
    for a, b in zip(route_nodes[:-1], route_nodes[1:]):
        dist_sum += distance_dict[points[a]][points[b]]
    total_distance += dist_sum
    trip_details[idx] = {'route_nodes': route_nodes, 'route_points': route_points, 'distance': dist_sum}
    print(f"Trip {idx}: {' -> '.join(route_points)}  distance={dist_sum:.4f} km")

print(f"\nTotal computed distance: {total_distance:.4f} km")

# -------------------------
# Enhanced constraint checks
# -------------------------
tol = 1e-6
constraint_violations = []

def sum_vars(var_list):
    return sum(vars_dict.get(v, 0.0) for v in var_list)

# incoming/outgoing checks
for k in range(1, n):
    incoming = [f"x_{i}_{k}" for i in range(n) if i != k]
    outgoing = [f"x_{k}_{j}" for j in range(n) if j != k]
    s_in = sum_vars(incoming)
    s_out = sum_vars(outgoing)
    ok_in = abs(s_in - 1) <= tol
    ok_out = abs(s_out - 1) <= tol
    print(f"Node {points[k]}: incoming_sum={s_in:.6f} ok_in={ok_in}; outgoing_sum={s_out:.6f} ok_out={ok_out}")
    if not ok_in:
        constraint_violations.append(f"{points[k]} incoming: {s_in} != 1")
    if not ok_out:
        constraint_violations.append(f"{points[k]} outgoing: {s_out} != 1")

# hospital departures/returns
dep = [f"x_0_{j}" for j in range(1, n)]
ret = [f"x_{i}_0" for i in range(1, n)]
s_dep = sum_vars(dep)
s_ret = sum_vars(ret)
print(f"Hospital departures sum={s_dep:.6f} expected={n_trips}")
print(f"Hospital returns    sum={s_ret:.6f} expected={n_trips}")
if abs(s_dep - n_trips) > tol:
    constraint_violations.append(f"Hospital departures: {s_dep} != {n_trips}")
if abs(s_ret - n_trips) > tol:
    constraint_violations.append(f"Hospital returns: {s_ret} != {n_trips}")

# capacity
s_total = sum_vars(list(unique_patient_incoming))
print(f"Total patient incoming visits: {s_total:.6f} capacity_rhs={global_capacity_rhs}")
if s_total > global_capacity_rhs + tol:
    constraint_violations.append(f"Capacity violated: {s_total} > {global_capacity_rhs}")

# singleton check
for k in range(1, n):
    s_single = vars_dict.get(f"x_0_{k}", 0.0) + vars_dict.get(f"x_{k}_0", 0.0)
    if s_single > 1 + tol:
        constraint_violations.append(f"Singleton loop for {points[k]} (x_0_{k} + x_{k}_0 = {s_single})")

# route endpoints check
hospital_idx = 0
for i, route in enumerate(routes):
    if route[0] != hospital_idx or route[-1] != hospital_idx:
        constraint_violations.append(f"Route {i+1} doesn't start/end at hospital: {[points[j] for j in route]}")

constraints_satisfied = len(constraint_violations) == 0
print(f"\nAll constraints satisfied? {constraints_satisfied}")
if not constraints_satisfied:
    print("Constraint violations:")
    for v in constraint_violations:
        print("  -", v)

# -------------------------
# Save predictions (active edges) to CSV
# -------------------------
results_dir = "results/physical"
os.makedirs(results_dir, exist_ok=True)
predictions_file = os.path.join(results_dir, "predictions.csv")
with open(predictions_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["from", "to", "var", "value"])
    for var, val in vars_dict.items():
        if var.startswith("x_"):
            _, i_str, j_str = var.split("_")
            writer.writerow([points[int(i_str)], points[int(j_str)], var, float(val)])

# -------------------------
# Save metrics & run summary & manifest
# -------------------------
metrics_entry = {
    "total_distance": total_distance,
    "constraints_satisfied": bool(constraints_satisfied),
    "constraint_violations": constraint_violations,
    "execution_time_sec": elapsed,
    "backend": backend_name,
    "shots": shots,
    "penalty": penalty_scale,
    "timestamp": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
}

metrics_file = os.path.join(results_dir, "metrics.json")
if os.path.exists(metrics_file):
    try:
        with open(metrics_file, "r") as f:
            existing_metrics = json.load(f)
        if not isinstance(existing_metrics, list):
            existing_metrics = [existing_metrics]
    except Exception:
        existing_metrics = []
else:
    existing_metrics = []
existing_metrics.append(metrics_entry)
with open(metrics_file, "w") as f:
    json.dump(existing_metrics, f, indent=2)

run_summary_entry = {
    "backend": backend_name,
    "shots": shots,
    "optimizer": "COBYLA",
    "reps": reps,
    "status": str(result.status),
    "objective_value": float(result.fval),
    "constraints_satisfied": bool(constraints_satisfied),
    "constraint_violations": constraint_violations,
    "total_distance": total_distance,
    "execution_time_sec": elapsed,
    "penalty": penalty_scale,
    "timestamp": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
}

run_summary_file = os.path.join(results_dir, "run_summary.json")
if os.path.exists(run_summary_file):
    try:
        with open(run_summary_file, "r") as f:
            existing_summary = json.load(f)
        if not isinstance(existing_summary, list):
            existing_summary = [existing_summary]
    except Exception:
        existing_summary = []
else:
    existing_summary = []
existing_summary.append(run_summary_entry)
with open(run_summary_file, "w") as f:
    json.dump(existing_summary, f, indent=2)

# Manifest
manifest_entry = {
    "team": "Team 4",
    "version": "1.0.0",
    "backend": backend_name,
    "min_qubits": 30,
    "shots": shots,
    "timestamp": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "penalty": penalty_scale,
    "artifacts": {
        "predictions": predictions_file,
        "metrics": metrics_file,
        "run_summary": run_summary_file,
        "loss_curve": os.path.abspath("qaoa_loss_curve_30qubits.png")
    },
    "evidence": {
        "job_ids_csv": job_ids_file,
        "logs_dir": logs_dir,
        "screenshots_dir": screenshots_dir
    }
}

manifest_file = "MANIFEST.json"
if os.path.exists(manifest_file):
    try:
        with open(manifest_file, "r") as f:
            existing_manifest = json.load(f)
        if not isinstance(existing_manifest, list):
            existing_manifest = [existing_manifest]
    except Exception:
        existing_manifest = []
else:
    existing_manifest = []
existing_manifest.append(manifest_entry)
with open(manifest_file, "w") as f:
    json.dump(existing_manifest, f, indent=2)

# -------------------------
# Loss curve plot (saved earlier if loss_history exists)
# -------------------------
if loss_history:
    plt.figure(figsize=(8,4))
    plt.plot(loss_history, marker='o')
    plt.title("QAOA Optimization Progress")
    plt.xlabel("Iteration")
    plt.ylabel("Energy")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, "qaoa_loss_curve_30qubits.png"))
    plt.show()

# -------------------------
# Summary printout
# -------------------------
print("\n" + "="*40)
print("RUN SUMMARY")
print("="*40)
print(f"Results written to: {results_dir}")
print(f"Predictions: {predictions_file}")
print(f"Metrics: {metrics_file}")
print(f"Run summary: {run_summary_file}")
print(f"Manifest appended to: {manifest_file}")
print(f"Constraints satisfied: {constraints_satisfied}")
if not constraints_satisfied:
    print("Violations:")
    for v in constraint_violations:
        print(" -", v)
print("="*40)


Quadratic program built.
Binary variables (should be 30): 30
Linear constraints: 18
QUBO created with penalty=170
QUBO binary variables: 33


qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-09-12 06:36:24,753: Default instance not set. Searching all available instances.
qiskit_runtime_service._discover_backends_from_instance:WARNING:2025-09-12 06:36:26,530: Invalid instance crn:v1:bluemix:public:quantum-computing:us-east:a/3ff804ab3cd148d684d3d710c3ed458b:a4282910-a8bb-4b34-950d-47d494c80026::


Using backend: ibm_torino (pending jobs: 419)
Running QAOA (MinimumEigenOptimizer) ...


RequestsApiError: '400 Client Error: Bad Request for url: https://quantum.cloud.ibm.com/api/v1/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different [execution mode](https://quantum.cloud.ibm.com/docs/guides/execution-modes).","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"902fa4ba-736a-4322-a272-e692cbf41d39"}\n'